# Implementación Path Relinking
---

In [108]:
from structure import instance, solution
from algorithms import grasp
from structure.solution import evaluate
from copy import deepcopy
from path_relinking import pathRelinking
import pandas as pd
import random
import os

In [109]:
path = "instances/MDG-a_12_100_m10.txt"
inst = instance.readInstance(path)
sol, conjunto_soluciones, tiempo_final_grasp = grasp.execute(inst, 1, 0.1)

In [110]:
conjunto_soluciones_pd = pd.DataFrame(conjunto_soluciones)
print(len(conjunto_soluciones_pd))
conjunto_soluciones_pd = conjunto_soluciones_pd.drop_duplicates("sol")

128


In [111]:
conjunto_soluciones_pd.sort_values("of", ascending=False)

,instance,sol,of
1,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 96, 13, 15, 51, 20, 86, 55, 59, 93}",354.25
60,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{98, 2, 90, 37, 59, 12, 20, 26, 91, 63}",351.82
102,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 70, 43, 79, 16, 20, 86, 59, 60, 93}",350.18
20,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{5, 70, 39, 8, 74, 43, 14, 79, 17, 60}",349.53
43,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{98, 5, 38, 39, 43, 77, 88, 60, 61, 62}",348.67
...,...,...,...
113,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{1, 4, 70, 8, 79, 83, 21, 23, 58, 95}",324.88
70,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{5, 73, 41, 28, 18, 22, 25, 26, 59, 60}",324.78
62,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{67, 3, 73, 11, 46, 84, 22, 94, 26, 30}",323.47
22,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{36, 7, 71, 75, 45, 78, 14, 21, 24, 31}",322.16


In [112]:
sol_max = conjunto_soluciones_pd[conjunto_soluciones_pd["of"] == max(conjunto_soluciones_pd["of"])].iloc[0,1]
conjunto_soluciones_pd["n_distintos"] = conjunto_soluciones_pd["sol"].map(lambda x: len(x.difference(sol_max)))
conjunto_soluciones_pd[conjunto_soluciones_pd["n_distintos"] > 0].sort_values("of", ascending=False)

,instance,sol,of,n_distintos
60,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{98, 2, 90, 37, 59, 12, 20, 26, 91, 63}",351.82,8
102,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 70, 43, 79, 16, 20, 86, 59, 60, 93}",350.18,5
20,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{5, 70, 39, 8, 74, 43, 14, 79, 17, 60}",349.53,10
43,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{98, 5, 38, 39, 43, 77, 88, 60, 61, 62}",348.67,10
10,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{98, 2, 37, 59, 12, 15, 20, 89, 91, 93}",348.22,6
...,...,...,...,...
113,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{1, 4, 70, 8, 79, 83, 21, 23, 58, 95}",324.88,10
70,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{5, 73, 41, 28, 18, 22, 25, 26, 59, 60}",324.78,9
62,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{67, 3, 73, 11, 46, 84, 22, 94, 26, 30}",323.47,10
22,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{36, 7, 71, 75, 45, 78, 14, 21, 24, 31}",322.16,10


A continuación, se realiza una implementación recursiva de Path Relinking entre dos soluciones específicas.

In [113]:
sol_0 = conjunto_soluciones[14]
sol_1 = conjunto_soluciones[57]

In [114]:
def transicion(sol_0, sol_1):
    best = None
    resultado = 0
    sol_2 = deepcopy(sol_0)
    sol_guia = deepcopy(sol_1)
    sol_guia["sol"] = sol_guia["sol"].difference(sol_0["sol"])
    for s0 in sol_0["sol"]:
        for s1 in sol_guia["sol"]:
            if s1 not in sol_0["sol"]:
                sol_2["sol"].remove(s0)
                sol_2["sol"].add(s1)
                sol_2["of"] = evaluate(sol_2)
                if best is None or sol_2["of"] > best["of"]:
                    anyadido = s1
                    best = deepcopy(sol_2)
                sol_2["sol"].remove(s1)
                sol_2["sol"].add(s0)
    
    sol_guia["sol"].remove(anyadido)

    if len(sol_guia["sol"]) > 1:
        resultado = transicion(best, sol_guia)

        if resultado["of"] < best["of"]:
            resultado = best
    else:
        print(best["sol"])
        resultado = best
    

    return resultado

best = transicion(sol_0, sol_1)

{97, 66, 98, 39, 10, 74, 82, 59, 93, 94}


In [115]:
best["of"]

341.01

In [116]:
sol_0["of"]

338.49

In [117]:
sol_1["sol"]

{13, 16, 25, 43, 59, 64, 70, 74, 79, 86}

### Opción para el Path Relinking con 1 instancia

La siguiente función es la versión final y general de PR paso a paso.

In [118]:
def relink(inicio, guia):
    current = deepcopy(inicio)
    best = deepcopy(inicio)

    to_add = guia["sol"] - current["sol"]
    to_remove = current["sol"] - guia["sol"]

    while to_add:
        vecinos = []
        for a in to_add:
            for r in to_remove:
                nuevo = deepcopy(current)
                nuevo["sol"].remove(r)
                nuevo["sol"].add(a)
                nuevo["of"] = evaluate(nuevo)
                vecinos.append(nuevo)
        if not vecinos:
            break
        current = max(vecinos, key=lambda s: s["of"])
        if current["of"] > best["of"]:
            best = deepcopy(current)

        to_add = guia["sol"] - current["sol"]
        to_remove = current["sol"] - guia["sol"]

    return best


Ahora se busca ejecutar PR entre todos los pares del conjunto élite. Mostramos cuáles son las mejores soluciones sin repetir y la mejor que encuentra.

In [119]:
# Seleccionamos las x mejores soluciones distintas (se puede modificar el nº de soluciones que se toman)
n_soluciones = 8
elite_df = conjunto_soluciones_pd.sort_values("of", ascending=False).drop_duplicates("sol").head(n_soluciones)
elite_solutions = elite_df.to_dict("records")

# Aplicamos Path Relinking entre todos los pares de soluciones
soluciones_pr = []
mejor_pr = None
for i in range(len(elite_solutions)):
    for j in range(i+1, len(elite_solutions)):
        s1 = elite_solutions[i]
        s2 = elite_solutions[j]
        pr1 = relink(s1, s2)
        pr2 = relink(s2, s1)
        soluciones_pr.extend([pr1, pr2])
        for cand in [pr1, pr2]:
            if mejor_pr is None or cand["of"] > mejor_pr["of"]:
                mejor_pr = cand

# Mostramos la mejor solución obtenida por Path Relinking
print("Mejor solución encontrada por Path Relinking:")
print("Conjunto:", mejor_pr["sol"])
print("Valor objetivo:", mejor_pr["of"])

# Convertimos a DataFrame para análisis posterior
df_pr = pd.DataFrame(soluciones_pr).drop_duplicates(subset="sol")
print("\nTop 5 soluciones PR:")
print(df_pr.sort_values("of", ascending=False).head(5))

print(f"\nTotal de soluciones generadas mediante PR: {len(df_pr)}")


Mejor solución encontrada por Path Relinking:
Conjunto: {64, 96, 13, 15, 51, 20, 86, 55, 59, 93}
Valor objetivo: 354.24999999999994

Top 5 soluciones PR:
                                             instance  \
0   {'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...   
14  {'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...   
26  {'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...   
36  {'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...   
44  {'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...   

                                         sol      of  n_distintos  
0   {64, 96, 13, 15, 51, 20, 86, 55, 59, 93}  354.25            0  
14   {98, 2, 26, 37, 91, 12, 20, 90, 59, 63}  351.82            8  
26  {64, 70, 43, 79, 16, 20, 86, 59, 60, 93}  350.18            5  
36    {5, 70, 39, 8, 74, 43, 14, 79, 17, 60}  349.53           10  
44   {98, 5, 38, 39, 43, 77, 88, 60, 61, 62}  348.67           10  

Total de soluciones generadas mediante PR: 7


### GRASP + Path Relinking con todas las instancias

En cada instancia:
* Se ejecutará GRASP.
* Se seleccionará el conjunto élite.
* Se ejecutará el Path Relinking.
* Se guardarán diversas métricas como la mejor solución GRASP, mejor solución PR, medias...

In [120]:
# Ruta a la carpeta con las 15 instancias
carpeta_instancias = "instances/"
archivos = sorted([carpeta_instancias + f for f in os.listdir(carpeta_instancias) if f.endswith(".txt")])

resultados = []

# Ejecutar GRASP+PR para cada archivo
for archivo in archivos:
    print(f"Procesando {archivo}...")

    try:
        mejor_pr, df_pr = pathRelinking(archivo, alpha=0.1, tiempo=60, elite_size=5)

        resultados.append({
            "instancia": os.path.basename(archivo),
            "mejor_objetivo_PR": mejor_pr["of"],
            "media_objetivos_PR": df_pr["of"].mean(),
            "desviacion_objetivos_PR": df_pr["of"].std(),
            "num_soluciones_PR": len(df_pr)
        })

    except Exception as e:
        print(f"Error procesando {archivo}: {e}")

# Guardar resultados en CSV
df_resultados = pd.DataFrame(resultados)
df_resultados.to_csv("resultados_path_relinking.csv", index=False)

print("Resultados guardados en 'resultados_path_relinking.csv'")


Procesando instances/MDG-a_10_100_m10.txt...
Procesando instances/MDG-a_12_100_m10.txt...
Procesando instances/MDG-a_13_n500_m50.txt...


KeyboardInterrupt: 